#### Getting started With Langchain And Open AI

In this quickstart we'll see how to:

- Get setup with LangChain, LangSmith and LangServe
- Use the most basic and common components of LangChain: prompt templates, models, and output parsers.
- Build a simple application with LangChain
- Trace your application with LangSmith
- Serve your application with LangServe

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
from langchain_openai import ChatOpenAI
# Use a cheaper default model to reduce likelihood of quota/billing issues.
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_retries=2)
print(llm)

e:\Gen AI\Langchain\1-Basics+Of+Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True} client=<openai.resources.chat.completions.completions.Completions object at 0x000002219E0C6420> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000221A0244680> root_client=<openai.OpenAI object at 0x00000221FBACDA90> root_async_client=<openai.AsyncOpenAI object at 0x00000221A002BA40> model_name='gpt-4o-mini' temperature=0.0 model_kwargs={} openai_api_key=SecretStr('**********') stream_usage=True max_retries=2


In [3]:
## Input and get response from LLM
import os
from openai import RateLimitError

question = "What is generative AI?"

try:
    result = llm.invoke(question)
except RateLimitError as e:
    # This usually means: the API key has no remaining credits / billing not enabled.
    result = None
    print("OpenAI request failed: HTTP 429 (insufficient_quota).")
    print("Fix: enable billing / add credits for your OpenAI API key, or use another provider.")
    print("Optional: set GROQ_API_KEY to auto-fallback to Groq.")

    if os.getenv("GROQ_API_KEY"):
        from langchain_groq import ChatGroq
        print("Falling back to Groq...")
        fallback_llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0)
        result = fallback_llm.invoke(question)
    else:
        print(f"Details: {e}")

OpenAI request failed: HTTP 429 (insufficient_quota).
Fix: enable billing / add credits for your OpenAI API key, or use another provider.
Optional: set GROQ_API_KEY to auto-fallback to Groq.
Details: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


In [4]:
print(result)

None


In [5]:
### Chatprompt Template
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert AI Engineer. Provide me answers based on the questions"),
        ("user","{input}")
    ]

)
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me answers based on the questions'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [6]:
## chain
import os
from openai import RateLimitError

chain = prompt | llm

payload = {"input": "Can you tell me about Langsmith?"}
response = None

try:
    response = chain.invoke(payload)
except RateLimitError as e:
    print("OpenAI request failed: HTTP 429 (insufficient_quota).")
    print("Fix: enable billing / add credits for your OpenAI API key, or use another provider.")
    print("Optional: set GROQ_API_KEY to auto-fallback to Groq.")

    if os.getenv("GROQ_API_KEY"):
        from langchain_groq import ChatGroq
        print("Falling back to Groq...")
        fallback_llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0)
        chain = prompt | fallback_llm
        response = chain.invoke(payload)
    else:
        print(f"Details: {e}")

print(response)

OpenAI request failed: HTTP 429 (insufficient_quota).
Fix: enable billing / add credits for your OpenAI API key, or use another provider.
Optional: set GROQ_API_KEY to auto-fallback to Groq.
Details: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
None


In [7]:
type(response)

NoneType

In [8]:
## stroutput Parser
import os
from openai import RateLimitError
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser

payload = {"input": "Can you tell me about Langsmith?"}
response = None

try:
    response = chain.invoke(payload)
except RateLimitError as e:
    print("OpenAI request failed: HTTP 429 (insufficient_quota).")
    print("Fix: enable billing / add credits for your OpenAI API key, or use another provider.")
    print("Optional: set GROQ_API_KEY to auto-fallback to Groq.")

    if os.getenv("GROQ_API_KEY"):
        from langchain_groq import ChatGroq
        print("Falling back to Groq...")
        fallback_llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0)
        chain = prompt | fallback_llm | output_parser
        response = chain.invoke(payload)
    else:
        print(f"Details: {e}")

print(response)

OpenAI request failed: HTTP 429 (insufficient_quota).
Fix: enable billing / add credits for your OpenAI API key, or use another provider.
Optional: set GROQ_API_KEY to auto-fallback to Groq.
Details: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
None
